# Biol 359A  | Simple and Multiple Linear Regression
### Spring 2023, Week 5
<hr>

Objectives:
-  Run and interpret a linear regression
-  Gain intuition about MLR results
-  Introduce concepts like overfitting and test data

__Question:__ Assume you run an experiment observing the impact of 10 different microbial soil populations on the viability of Arabidopsis thaliana (plant). How many 2-sampled t-tests would you need to run to evaluate pairwise effect (or lack thereof) of the different soil populations?  

$$ 10 \times (10-1) / 2  = 45 $$

__Question:__ Assume you run an experiment observing the impact of 10 different microbial soil populations on the viability and growth of Arabidopsis thaliana (plant model).

Provided an alpha=0.05 as the threshold for your study, what is your bonferoni adjusted alpha for each 2-sample t-test? 

$$\alpha/n_{tests} = 0.05/45 =  0.0011$$

__False:__ The null distribution is the same for all null hypotheses.

-  __T-Test__: Mean/Variance = _t_-distribution

-  __ANOVA__: Variance/Variance = _F_-distribution

In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr23/week5_linearregression.git
!mkdir ./data
!cp week5_linearregression/data/* ./data
!cp week5_linearregression/clean_data.py ./

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt
import ipywidgets as widgets

from scipy.stats import ttest_ind as ttest
from scipy import stats
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

%matplotlib inline


TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (12,12)
COLORS= ["#008080","#CA562C"]

sns.set_context("notebook")
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
sns.set(font_scale=1, rc={'figure.figsize':FIG_SIZE}) 

### We are going to use the Wisconsin Diagnostic Breast Cancer dataset once again

From the data source: Wisconsin Diagnostic Breast Cancer (WDBC)

```
	Features are computed from a digitized image of a fine needle
	aspirate (FNA) of a breast mass.  They describe
	characteristics of the cell nuclei present in the image.
	A few of the images can be found at
	http://www.cs.wisc.edu/~street/images/

	Separating plane described above was obtained using
	Multisurface Method-Tree (MSM-T) [K. P. Bennett, "Decision Tree
	Construction Via Linear Programming." Proceedings of the 4th
	Midwest Artificial Intelligence and Cognitive Science Society,
	pp. 97-101, 1992], a classification method which uses linear
	programming to construct a decision tree.  Relevant features
	were selected using an exhaustive search in the space of 1-4
	features and 1-3 separating planes.

	The actual linear program used to obtain the separating plane
	in the 3-dimensional space is that described in:
	[K. P. Bennett and O. L. Mangasarian: "Robust Linear
	Programming Discrimination of Two Linearly Inseparable Sets",
	Optimization Methods and Software 1, 1992, 23-34].
    
    Source:
    W.N. Street, W.H. Wolberg and O.L. Mangasarian 
	Nuclear feature extraction for breast tumor diagnosis.
	IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science
	and Technology, volume 1905, pages 861-870, San Jose, CA, 1993.
```

What do all the column names mean?

- ID number
- Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry 
- fractal dimension ("coastline approximation" - 1) - a measure of "complexity" of a 2D image.


Cateogory Distribution: 357 benign, 212 malignant

# Question 1-2: Characteristics of the dataset

Answer some questions about the structure of our data! 

In [ ]:
import clean_data

original_cancer_dataset = clean_data.generate_clean_dataframe()
cancer_dataset = original_cancer_dataset.reset_index()
cancer_dataset.drop("ID", axis=1, inplace=True)
cancer_dataset

# Question 3-5: Simple Linear Regression

We will start with the scatter plots that we used to discuss correlation during Week 2, now through the perspective of using a Simple Linear Regression to characterize these relationships. We are going to introduce the concept of the __Coefficient of Determination__: $R^2$. 

$$ R^2 = \frac{SSR}{SST} = 1 - \frac{SSE}{SST}$$

Where 
- SSE = error of sum of squares
- SSR = regression sum of squares
- SST = total sum of squares

This value conceptually represents the "amount of variance in y is explained by x" and can be thought of as how well the model fits the training data. This is a seperate concept to the Pearson Correlation Coefficient, $\rho$, which in the cases of a __strictly linear regression__, $R^2 = \rho^2$. 

You can also calculate the adjusted $R^2$ which accounts for the model size and parameters. This way, you can compare the adjusted $R^2$ of simple models to the adjusted $R^2$ of complex models, which you can't necessarily do if you just use  $R^2$. 

$$R^2_{adj} = 1 - \frac{\frac{SSE}{n-k-1}}{\frac{SST}{n - 1}}$$

Where 
- n = number of data points
- k number of independent variables

Generally speaking, $R^2$ is used to characterize the fit of a model, where as $\rho$ is used to characterize the relationship between two variables. 

We will perform a simple linear regression between the response variable, y, and the feature, x. 

#### In order for us to use diagnosis as a variable, we need to turn it from categorical (M, B) into numerical values. We will set "M" to 1, and "B" to 0. 

In [ ]:
cancer_dataset['diagnosis'].replace(["M","B"], [1,0], inplace=True)
cancer_dataset

In [ ]:
# Create scatter plots of the various features
def calculate_correlation(x,y):
    """calculate pearson correlation"""
    return calculate_mean((x - calculate_mean(x)).transpose() * (y - calculate_mean(y))) / np.sqrt(calculate_variance(x) * calculate_variance(y))

def calculate_mean(x):
    """get the sample mean"""
    return np.sum(x) / len(x)

def calculate_variance(x):
    """calculate variance of the dataset"""
    return calculate_mean((x - calculate_mean(x))**2)

def simple_linear_regression(x,y, ax):
    regression = linear_model.LinearRegression() 
    regression.fit(x.values.reshape(-1,1),y)
    coef_of_determination = regression.score(x.values.reshape(-1,1),y)
    x_range = np.linspace(min(x), max(x))
    plt.plot(x_range, regression.coef_*x_range + regression.intercept_, "--",)
    plt.text(1.01, 0.98, r"$\beta_1 = {0:.2f}$".format(regression.coef_[0]),
             ha='left', va='top', size =LABEL_FONT,
             transform=ax.transAxes)
    plt.text(1.01, 0.95, r"$\beta_0 = {0:.2f}$".format(regression.intercept_),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.text(1.01, 0.92, r"$R^2 = {0:.2f}$".format(coef_of_determination),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    
@widgets.interact(x=list(cancer_dataset), y=list(cancer_dataset))    
def make_scatterplot(x="mean_radius",y="mean_area"):
    """make scatterplot with correlation value and regplot"""
    colors=["#e28743", "#1e81b0"]
    corr = calculate_correlation(cancer_dataset[x], cancer_dataset[y])
    index = int(corr > 0.5)
    plt.title(r"correlation: $\rho = ${:.3f}".format(corr), color= "grey", size=TITLE_FONT)
    df = cancer_dataset.reset_index()
    ax = sns.scatterplot(data=df, x=x, y=y, alpha=0.4, hue="diagnosis")
    simple_linear_regression(cancer_dataset[x], cancer_dataset[y], ax)


### Aside: Statistical tests and beta coefficients

We did not spend a lot of time discussing statistical tests with respect to associated p-values, but the interpretation of these values are the same as we've discussed throughout the quarter. You can use hypothesis testing to interrogate your regression coefficients. Here the most common test is using the null hypothesis of $\mathcal{H}_0: \beta_i = 0$, which corresponds to the _t_-statistic. If you do this test on __binary__ or __boolean__ variable, you get the same exact result as a t-test.

In [ ]:

def run_ind_ttest(feature="mean_symmetry", dataset = original_cancer_dataset, welch=False):
    """
    run two sample t-tests
    For the motivated, visit 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html 
    to see the documentation to see how to write the code for a t-test."""
    cat1 = dataset.xs("M", level=1)
    cat2 = dataset.xs("B", level=1)
    df = dataset.reset_index()
    f, (ax_hist, ax_box) = plt.subplots(2, sharex=True)

    sns.boxplot(data=df, x=feature, y=df["diagnosis"], ax=ax_box)
    sns.histplot(data=df, x=feature, hue=df["diagnosis"], ax=ax_hist, stat="probability", kde=True)
    if welch: tstat, pvalue = ttest(cat1[feature], cat2[feature], equal_var=False)
    else: tstat, pvalue = ttest(cat1[feature], cat2[feature])
    print(f"p-value: {pvalue:.2e}")
    plt.show()
    make_scatterplot("diagnosis", feature)
    slope, intercept, r_value, reg_p_value, std_err = stats.linregress(cancer_dataset["diagnosis"], cancer_dataset[feature])

    print(f"p-value: {reg_p_value:.2e}")    
    
run_ind_ttest()



In [ ]:
run_ind_ttest(feature="mean_fractal_dimension")

# Questions 6-8: Multiple Linear Regression 

We have other variables that we can use to explore the relationships in this dataset. We can use multiple linear regression to include more variables, where we are drawing a multi-dimensional shape, rather than simply a line, as our model. The plot below is currently the same as the one we looked at above with mean_area and mean_radius. Try adding some other variables into your regression by checking the boxes.

In [ ]:
def linear_regression(df, feature_cols, response_col, standardized = False):
    """
    Use linear_model to run a linear regression using sklearn
    
    """
    X = df[feature_cols]
    y = df[response_col]
    if standardized:
        X = StandardScaler().fit_transform(X)
        y = StandardScaler().fit_transform(y.values.reshape(-1, 1))
    regression = linear_model.LinearRegression() 
    regression.fit(X,y)
    
    try:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_))
    except TypeError:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_[0]))
    print('Regression Coefficients: ')
    for feature, coef in zip(feature_cols, regression.coef_.flatten()):
        print(f'{feature} ~ {coef:.2f}')
    return regression.predict(X), regression.score(X,y)

def error_distribution(true, pred, hue=None):
    ax = plt.subplots(figsize=(12, 8))
    error = true-pred
    if hue is not None: 
        sns.kdeplot(error, hue=hue, shade=True, alpha=.2)
    else:
        sns.kdeplot(error, shade=True, alpha=.2)
    sns.despine()

    plt.xlabel(r'Residuals ($\epsilon$)')
    plt.title('Error', size=TITLE_FONT)
    plt.show()
    
def parity_plot(true, pred, r_squared=None, title='', hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(x=true, y=pred, hue=hue)
    else: 
        sns.scatterplot(x=true, y=pred)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    ax.set_box_aspect(1)
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()
    
    # error_distribution(true, pred, hue)


def run_regression(feature_cols = 
                   ['mean_radius', 
                    'mean_texture', 
                    'mean_perimeter', 
                    'mean_smoothness', 
                    'mean_compactness', 
                    'mean_concavity', 
                    'mean_concave_points', 
                    'mean_symmetry', 
                    'mean_fractal_dimension'], 
                     response_col='mean_area',
                     standardized=False,
                     parity=True):
    y_pred, r_squared = linear_regression(cancer_dataset, feature_cols, response_col, standardized = standardized)
    if parity: parity_plot(cancer_dataset[response_col], y_pred.flatten(), r_squared, hue=cancer_dataset["diagnosis"])


@widgets.interact(response=list(cancer_dataset))   
def regression_wrapper(response="mean_radius",
                       diagnosis=False,
                       radius=False, 
                       texture=False, 
                       perimeter=False, 
                       area=True,
                       smoothness=False, 
                       compactness=False, 
                       concavity=False, 
                       concave_points=False, 
                       symmetry=False, 
                       fractal_dimension=False):
    features = []
    if diagnosis: features.append("diagnosis")
    if radius: features.append("mean_radius")
    if texture: features.append("mean_texture")
    if perimeter: features.append("mean_perimeter")
    if area: features.append("mean_area")
    if smoothness: features.append("mean_smoothness")
    if compactness: features.append("mean_compactness")    
    if concavity: features.append("mean_concavity")
    if concave_points: features.append("mean_concave_points")
    if symmetry: features.append("mean_symmetry")
    if fractal_dimension: features.append("mean_fractal_dimension")
        

    run_regression(feature_cols=features, response_col = response)

# Question 9: Normalization

We need to be careful about how we interpret parameters in relation to each other. One method that I can use to compare coefficients is to __standardize__ the features. This will effectively shift the distribution of all of our features to have a Standard Normal ($\mu=0,\sigma^2=1$) distribution. We can then compare the coefficients to each other, and interpret the magnitude within the model. 

Note: This transformation prevents you from making a direct interpretation of the coefficient as it relates to the actual value, and is now unitless. You should interpret these as being "model units". This transformation also means that the y-intercept will be zero. 

In [ ]:
@widgets.interact(response=list(cancer_dataset))   
def regression_wrapper(response="mean_area", 
                       radius=True, 
                       texture=True, 
                       perimeter=True, 
                       area=False,
                       smoothness=True, 
                       compactness=True, 
                       concavity=True, 
                       concave_points=True, 
                       symmetry=True, 
                       fractal_dimension=True):
    features = []
    if radius: features.append("mean_radius")
    if texture: features.append("mean_texture")
    if perimeter: features.append("mean_perimeter")
    if area: features.append("mean_area")
    if smoothness: features.append("mean_smoothness")
    if compactness: features.append("mean_compactness")    
    if concavity: features.append("mean_concavity")
    if concave_points: features.append("mean_concave_points")
    if symmetry: features.append("mean_symmetry")
    if fractal_dimension: features.append("mean_fractal_dimension")
        

    run_regression(feature_cols=features, response_col = response, standardized=True, parity=False)

### You can ignore the code below and skip to the relevant example for question 11.

In [ ]:
def quadratic(x):
    # Default - 2x^2 + 2
    return 2*x**2 + 2

def generate_noisy_data(function, noise_std, n=10, measurement_std=.2, initial_value=0, x_max=3, plot=True):
    """
    This function generates noisy data with a certain amount of error applied to the function response.
    The error is normally distributed around the noise_std.
    """
    x = np.linspace(0, x_max, n) 
    x_noise = np.random.normal(0, measurement_std, len(x))
    x += x_noise
    y_noise = np.random.normal(0, noise_std, len(x))
    y = function(x) + initial_value
    y += y_noise
    if plot:
        plt.plot(x, y, 'C0.', label='data')
        x_func = np.linspace(0, max(x)+measurement_std)
        y_func = function(x_func) + initial_value
        plt.plot(x_func, y_func, 'C0--', label='function')
        plt.fill_between(x_func, y_func+noise_std, y_func-noise_std,
                         alpha=0.1)          # Transparency of the fill
        plt.title(r'$ y = 2x^2 + 2$ with noise (std of {})'.format(noise_std))
        plt.legend(loc='best')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.xlim(0, max(x)+measurement_std)
        plt.show()
    return x, y


def plot_model(x, y, x_model, y_model, title = '',r_squared=None, x_test=None, y_test=None):
    """
    Plotter function.
    """
    
    plt.plot(x,y, 'o', label='data')
    if x_test is not None: plt.plot(x_test,y_test,"o", label='test')
    plt.plot(x_model, y_model, '--', label='model')
    plt.legend(loc='best')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(0, max(x))
    plt.title(title)
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=plt.gca().transAxes)    
    plt.show()


def polynomial_feature_example(x, y, degrees=6, x_test=None, y_test=None):
    """
    Perform regularization on a polynomial feature set. 
    """
    poly_transform = PolynomialFeatures(degree=degrees, include_bias = False)
    x_poly = poly_transform.fit_transform(x.reshape(-1,1))
    
    #Regularization techniques need to be scaled in order to work properly
    x_scaler = StandardScaler().fit(x_poly)
    y_scaler = StandardScaler().fit(y.reshape(-1,1))
    x_poly_z = x_scaler.transform(x_poly)
    y_z = y_scaler.transform(y.reshape(-1,1))
    
    #Code to perform the model fitting and parameter estimation
    #Least Squares problem
    plt.suptitle('Linear Regression', fontsize=20, fontweight='bold')
    lm_poly = linear_model.LinearRegression(fit_intercept=True)
    lm_poly.fit(x_poly_z,y_z)

    x_model = np.linspace(min(x), max(x), 150).reshape(-1,1)
    x_model_transform = poly_transform.fit_transform(x_model)
    x_model_transform_z = x_scaler.transform(x_model_transform)
    
    
    y_model = lm_poly.predict(x_model_transform_z)*y_scaler.scale_ + y_scaler.mean_
    
    #********************************************************************************
    # Coefficients from scaled model can be transformed back into original units
    # This code is outside the scope of this class and can be ignored. 
    
    unscaled_coefficients = (lm_poly.coef_ * y_scaler.scale_ / x_scaler.scale_).flatten()
    
    poly_terms = [r'$({0:.3f})x^{{{1}}}$'.format(coef, i+1) for i, coef in enumerate(unscaled_coefficients)
                 if coef != 0]
    
    unscaled_intercept = lm_poly.intercept_*y_scaler.scale_ + y_scaler.mean_ \
                            - sum(unscaled_coefficients*x_scaler.mean_)
        
    intercept_str = r'${0:.1f} + $'.format(unscaled_intercept[0])
    title =  intercept_str + r'$+$'.join(poly_terms)
    #********************************************************************************
    r_squared = lm_poly.score(x_poly_z,y_z)
    ax = plot_model(x, y, x_model, y_model, title=title, r_squared = r_squared, x_test=x_test, y_test=y_test)


# Questions 10: Underdefined and Overdefined

In class we discussed two different types of problems where the number of features does not equal the number of observations. We are going to generate data from a quadratic equation with some noise, and then we are going to try and fit a new linear model:

$$ y = \beta_0 + \sum_{i=1}^{degrees} \beta_i x^i$$ 

Where we are going to take an original dataset of 1 feature, and generate multiple new features from the original feature. From this equation, if we set `degrees` equal to 1, we are doing a simple linear regression. 

If we set `degrees` equal to 2, we are fitting a MLR model with $x$ and $x^2$ as our features. 
Creating polynomial features is one way that we can do __feature engineering__, a common activity in machine learning.
Another benefit of using polynomial features is so that we can represent all of the data and the model in a 2 dimensional plot. 


Answer the questions in the classword regarding the two different types of problems:

In [ ]:
@widgets.interact_manual(n=(0,100), degrees=(1,12))
def run_polynomial_experiment(n=20, degrees=1):
    x_data, y_data = generate_noisy_data(quadratic, 2, n, plot=False)
    polynomial_feature_example(x_data, y_data, degrees=degrees)

### Optional: A little bit more about feature engineering
When it comes to Machine Learning, you don't always have the features that you want to make the best model. This concept is closely tied to the ideas of Neural Networks (a core part of Machine Learning, which is outside the scope of this class) where the goal is to __extract__ the appropriate features for a model. However, if we want to have control over what types of features we put into our model (in order for us to know explicitly how the model works) we can use feature engineering. 

We already covered one aspect of feature engineering in the classwork, specifically polynomial features. There are two other techniques that would have been appropriate for this dataset. Let's tackle the following problem: The model was not good at the extremes of the dataset, and it seems like we could draw two lines that were better than one line. 

__One-Hot Encoding__: We can change the `diagnosis` feature into two features: `malignant` and `benign` and assign a value of 1 or 0 to each feature. These features would be __mutually exclusive__, meaning that both features cannot have a value of 1.

__Interactions__: We could multiply every feature by every other feature (or selection of features) to add interactions in the model. For example:

$$
X = 
\begin{bmatrix}
x_{1,1} & x_{1,2} & x_{1,3}\\
x_{2,1} & x_{1,2} & x_{2,p}\\
\vdots & \dots & \vdots\\ 
x_{n,1} & x_{n,2} & x_{n,}\\
\end{bmatrix}
$$

$$
X_{interactions} = 
\begin{bmatrix}
x_{1,1}*x_{1,2} & x_{1,1}*x_{1,3} & x_{1,2}*x_{1,3}\\
x_{2,1}*x_{2,2} & x_{2,1}*x_{2,3} & x_{2,2}*x_{2,3}\\
\vdots & \dots & \vdots\\ 
x_{n,1}*x_{n,2} & x_{n,1}*x_{n,3} & x_{n,2}*x_{n,3}\\
\end{bmatrix}
$$


Try to consider how combining these two feature engineering techniques would effectively make two different models, one when `malignant` is equal to 1, and one when `benign` is equal to 1. 



### Optional: MLR by hand

For those motivated to understand MLR a little deeper, some code to run an MLR by hand is shown below.

There is an extra step in here that we didn't focus on in class that's required to fit the intercept.
Let's start with a feature matrix, $X$, and a response vector, $Y$:

$$
Y = 
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots\\ 
y_n\\
\end{bmatrix}
$$


$$
X = 
\begin{bmatrix}
x_{1,1} & \dots & x_{1,p}\\
x_{2,1} & \dots & x_{2,p}\\
\vdots & \dots & \vdots\\ 
x_{n,1} & \dots & x_{n,p}\\
\end{bmatrix}
$$

We will pad the $X$ with a column of $1$'s before we solve the linear regression problem: 

$$
\tilde{X} = 
\begin{bmatrix}
1&x_{1,1} & \dots & x_{1,p}\\
1&x_{2,1} & \dots & x_{2,p}\\
1&\vdots & \dots & \vdots\\ 
1&x_{n,1} & \dots & x_{n,p}\\
\end{bmatrix}
$$

If you want to fit an intercept:
$$
B = (\tilde{X}^T\tilde{X})^{-1}(\tilde{X}^TY)
$$

Where B: 
$$
B = 
\begin{bmatrix}
\beta_0 \\
\beta_1 \\
\vdots\\ 
\beta_n\\
\end{bmatrix}
$$ 

Therefore, when you solve for $\beta_0$, the feature that it's using is a column of 1, which means that it is a constant. 